Knowledge Graph-based Inference Engine for Thoracic Aortic Aneurysm

In [1]:
import pandas as pd
import os
import sys         
sys.path.insert(0, 'C:/bmin520/python')        
from utils import (load_nodes,load_edges,filter_nodes_by_keywords,build_graph,get_nodes_within_distance,subset_graph,save_subgraph)

# Set  project directory (This was programmed on a Windows machine)
proj_dir = 'C:/bmin520'
os.chdir(proj_dir)

# Directories
primekg_dir = os.path.join(proj_dir, "PrimeKG/datasets/data")
pmbb_dir = os.path.join(proj_dir, "PMBB-Release-2020-2.3")
kg_file = os.path.join(primekg_dir, "kg.csv")
nodes_file = os.path.join(primekg_dir, "nodes.csv")
icd9_file = os.path.join(pmbb_dir, "pmbb_icd-9-matrix.txt")
icd10_file = os.path.join(pmbb_dir, "pmbb_icd-10-matrix.txt")
# variants_file = os.path.join(pmbb_dir, "pmbb_variants.txt")

# Read and summarize PrimeKG data
primekg = pd.read_csv(kg_file, low_memory=False)
primekg.query('y_type=="disease"|x_type=="disease"')

,relation,display_relation,x_index,x_id,x_type,x_name,x_source,y_index,y_id,y_type,y_name,y_source
346728,contraindication,contraindication,15193,DB05271,drug,Rotigotine,DrugBank,33577,5044,disease,hypertensive disorder,MONDO
346729,contraindication,contraindication,15193,DB05271,drug,Rotigotine,DrugBank,36035,1200_1134_15512_5080_100078,disease,hypertension,MONDO_grouped
346730,indication,indication,16687,DB00492,drug,Fosinopril,DrugBank,33577,5044,disease,hypertensive disorder,MONDO
346731,indication,indication,16687,DB00492,drug,Fosinopril,DrugBank,36035,1200_1134_15512_5080_100078,disease,hypertension,MONDO_grouped
346732,contraindication,contraindication,14483,DB13956,drug,Estradiol valerate,DrugBank,33577,5044,disease,hypertensive disorder,MONDO
...,...,...,...,...,...,...,...,...,...,...,...,...
6499510,exposure_disease,linked to,31393,11565,disease,metabolic syndrome X,MONDO,61825,D015032,exposure,Zinc,CTD
6499511,exposure_disease,linked to,28208,5148,disease,type 2 diabetes mellitus,MONDO,61825,D015032,exposure,Zinc,CTD
6499512,exposure_disease,linked to,84172,1076,disease,glucose intolerance,MONDO,61825,D015032,exposure,Zinc,CTD
6499513,exposure_disease,linked to,33063,100130,disease,adult acute respiratory distress syndrome,MONDO,61825,D015032,exposure,Zinc,CTD


Subset KG to thoracic aortic aneurysm-related nodes and edges

In [3]:
# Check if the subgraph directory exists and create it if not
subgraph_data_dir = os.path.join(proj_dir, "subgraph")
os.makedirs(subgraph_data_dir, exist_ok=True)

nodes = load_nodes(primekg_dir)
edges = load_edges(primekg_dir)
g = build_graph(nodes, edges)

# keywords to subset nodes by (courtesy of Teddy)
keywords = ["aortic aneurysm, familial thoracic", "thoracic aortic aneurysm", 
            "aortic aneurysm, thoracic", "aneurysm of sinus of Valsalva", 
            "aortic aneurysm (disease)", "aneurysm or dilation of ascending aorta",
            "congenital aneurysms of the great vessels", "Thoracic aortic aneurysm", 
            "Ascending tubular aorta aneurysm", "Descending thoracic aorta aneurysm",
            "Aortic arch aneurysm", "Aortic root aneurysm",
            "Fusiform descending thoracic aortic aneurysm", "Saccular descending thoracic aortic aneurysm",
            "Fusiform ascending tubular aorta aneurysm", "Fusiform aortic arch aneurysm", 
            "Saccular aortic arch aneurysm"]

keyword_nodes = filter_nodes_by_keywords(nodes, keywords)

# Get nodes within max_distance from the keyword-related nodes using BFS
start_nodes = keyword_nodes['node_index'].tolist()
nodes_to_keep = get_nodes_within_distance(g, start_nodes, max_distance=1)

# Subset graph and save
sub_nodes, sub_edges = subset_graph(nodes, edges, nodes_to_keep)
out_dir = subgraph_data_dir
save_subgraph(sub_nodes, sub_edges, out_dir)

print("Subgraph creation complete. Files saved.")

C:\bmin520/python\utils.py:24: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask = nodes['node_name'].str.contains('|'.join(keywords), case=False, na=False)


Subgraph creation complete. Files saved.


In [6]:
drug_features_file = os.path.join(primekg_dir, "drug_features.csv")
disease_features_file = os.path.join(primekg_dir, "disease_features.csv")

# Load feature files and subset
drug_features = pd.read_csv(drug_features_file)
disease_features = pd.read_csv(disease_features_file)
subset_drug_features = drug_features[drug_features['node_index'].isin(nodes_to_keep)]
subset_disease_features = disease_features[disease_features['node_index'].isin(nodes_to_keep)]

# Save subsetted features
subset_drug_features_file = os.path.join(subgraph_data_dir, "subset_drug_features.csv")
subset_disease_features_file = os.path.join(subgraph_data_dir, "subset_disease_features.csv")
subset_drug_features.to_csv(subset_drug_features_file, index=False)
subset_disease_features.to_csv(subset_disease_features_file, index=False)

In [4]:
import os
import pandas as pd

data_dir = os.path.join(proj_dir, "PrimeKG/datasets/data")

# List of key files and description (from PrimeKG repository)
files_to_check = [
    ("nodes.csv", "Node-level information."),
    ("edges.csv", "Edges representing undirected relationships between nodes."),
    ("kg.csv", "The Precision Medicine knowledge graph (joined nodes and edges)."),
    ("disease_features.csv", "Textual descriptions of diseases."),
    ("drug_features.csv", "Textual descriptions of drugs."),
    ("kg_raw.csv", "Intermediate KG from joining nodes and edges."),
    ("kg_giant.csv", "Intermediate KG after taking largest connected component."),
    ("kg_grouped.csv", "Intermediate KG after grouping diseases."),
    ("kg_grouped_diseases.csv", "List of all diseases and their assigned group name."),
    ("kg_grouped_diseases_bert_map.csv", "Manual grouping for diseases using BERT model.")
]18

for fname, desc in files_to_check:
    fpath = os.path.join(data_dir, fname)
    if os.path.exists(fpath):
        df = pd.read_csv(fpath, nrows=3)  # just read a few rows
        print(f"File: {fname}")
        print(f"Description: {desc}")
        print("Columns:", df.columns.tolist())
        print("First few rows:")
        if len(df.head()) > 100:
            short = df.head()[:100] + '...'
            print(short, "\n")

        else:
            print(df.head(), "\n")
    else:
        print(f"File not found: {fname}\n")


File: nodes.csv
Description: Node-level information.
Columns: ['node_index', 'node_id', 'node_type', 'node_name', 'node_source']
First few rows:
   node_index  node_id     node_type node_name node_source
0           0     9796  gene/protein    PHYHIP        NCBI
1           1     7918  gene/protein    GPANK1        NCBI
2           2     8233  gene/protein     ZRSR2        NCBI 

File: edges.csv
Description: Edges representing undirected relationships between nodes.
Columns: ['relation', 'display_relation', 'x_index', 'y_index']
First few rows:
          relation display_relation  x_index  y_index
0  protein_protein              ppi        0     8889
1  protein_protein              ppi        1     2798
2  protein_protein              ppi        2     5646 

File: kg.csv
Description: The Precision Medicine knowledge graph (joined nodes and edges).
Columns: ['relation', 'display_relation', 'x_index', 'x_id', 'x_type', 'x_name', 'x_source', 'y_index', 'y_id', 'y_type', 'y_name', 'y_sourc

In [2]:
import sys
sys.path.insert(0, 'C:/bmin520/python')
from utils import filter_pmbb_by_icd, enforce_uniform_sampling
import pandas as pd

pmbb_dir = "C:/bmin520/PMBB-Release-2020-2.3"
output_dir = "C:/bmin520/subgraph/patients_filtered_new"
icd9_code = '441.2'
icd10_code = 'I71.2' #TAA, not all have icd9, most have icd10
num_patients = 1842
seed = 1

sampled_ids_file = filter_pmbb_by_icd(icd9_code, icd10_code, pmbb_dir, output_dir, num_patients, seed)

sampled_ids = pd.read_csv(sampled_ids_file)['PMBB_ID'].tolist()
enforce_uniform_sampling(pmbb_dir, sampled_ids, output_dir)


KeyboardInterrupt: 

In [4]:
# Filter icd10 matrix file by sampled patient IDs file and return and save the filtered matrix

import pandas as pd
import os

# Paths
pmbb_dir = "C:/bmin520/PMBB-Release-2020-2.3"
output_dir = "C:/bmin520/subgraph/patients_filtered_new"
sampled_ids_file = os.path.join(output_dir, "sampled_patient_ids.csv")
icd10_file = os.path.join(pmbb_dir, "PMBB-Release-2020-2.3_phenotype_icd-10-matrix.txt")

sampled_ids = pd.read_csv(sampled_ids_file)['PMBB_ID'].tolist()

# Filter ICD-10 matrix
icd10_data = pd.read_csv(icd10_file, sep='\t')
filtered_icd10 = icd10_data[icd10_data['PMBB_ID'].isin(sampled_ids)]

# Save
output_file = os.path.join(output_dir, "filtered_icd10_matrix.csv")
filtered_icd10.to_csv(output_file, index=False)


In [ ]:
# Come back to later, currently not useful.


import pandas as pd
from pyplink import PyPlink

# Paths
plink_file_prefix = "C:/bmin520/PMBB-Release-2020-2.0_genetic_exome_GL"
sampled_ids_file = "C:/bmin520/subgraph/patients_filtered/sampled_patient_ids.csv"

aortopathy_genes = ["ACTA2", "ADAMTS10", "BGN", "CBS", "COL3A1",
                    "COL5A1", "COL5A2", "EFEMP2","FBN1", "FBN2",
                    "FLNA", "FOXE3", "LOX", "MED12"]

sampled_ids = pd.read_csv(sampled_ids_file)['PMBB_ID'].tolist()

# Load Plink data
pyp = PyPlink(plink_file_prefix)
fam_data = pyp.get_fam()
filtered_fam = fam_data[fam_data['iid'].isin(sampled_ids)]
bim_data = pyp.get_bim()
bim_data['gene'] = bim_data['snp'].str.split(':').str[-1]
filtered_bim = bim_data[bim_data['gene'].isin(aortopathy_genes)]
lof_results = []

for marker in filtered_bim['snp']:
    genotypes = pyp.get_geno_marker(marker) 
    for idx, genotype in enumerate(genotypes):
        patient_id = fam_data.iloc[idx]['iid']
        if genotype in [1, 2]:  # Heterozygous or homozygous var allele
            lof_results.append({
                "PMBB_ID": patient_id,
                "Gene": filtered_bim.loc[filtered_bim['snp'] == marker, 'gene'].values[0],
                "Marker": marker,
                "Genotype": genotype
            })

lof_results_df = pd.DataFrame(lof_results)
output_file = "C:/bmin520/subgraph/patients_filtered/lof_results.csv"
lof_results_df.to_csv(output_file, index=False)